# Quantum Computation - Week 9

&nbsp;

## Contents

&nbsp;

1. [Variational Quantum Eigensolver](#VQE)
    
    1.1 [Implementation with Aqua](#imp)
    


2. [Quantum Approximate Optimization Algorithm](#QAOA)

&nbsp;

# 1. Variational Quantum Eigensolver <a id='VQE'></a>

&nbsp;

In many applications it is important to find the minimum eigenvalue of a matrix. For example, in chemistry, the minimum eigenvalue of a Hermitian matrix characterizing the molecule is the ground state energy of that system; VQE may also be used dor combinatorial optimization problems (such as the Traveling Salesman).

In the future, the quantum phase estimation algorithm may be used to find the minimum eigenvalue. However, its implementation on useful problems requires circuit depths exceeding the limits of hardware available in the NISQ era. 

&nbsp;

### Intuition

&nbsp;

Real world problems such as social network interactions and marketing strategies may be modeled with graphs composed of vertices and edges.

Consider a simple graph coloring puzzle that involves filling in some of its vertices with one color, and the rest of its vertices with another color. Solving the puzzle successfully requires achieving the highest possible score, which is defined as the total of the numbers (weights) on the edges connecting vertices that have different colors.

<img src="img\max_cut.png" width="500 px" align="center">


Coloring the vertices of a weighted graph with two colors is one way of expressing the $MaxCut$ problem, in which the score is calculated by adding up the weights on the edges that are cut by a line drawn between vertices of different colors.

The general equation for calculating the energy of a given coloring is a simple difference (up to a normalization factor) of $ws$, the total weight between same color vertices, and $wd$, the total weight between different color vertices.

$$ e = \frac{1}{2}(w_s - w_d) $$

The $2^3$ coloring combinations can be represented with basis states using 3 qubits, with $| 0 \rangle$ representing red coloring and $|1 \rangle$ representing blue coloring.

The energy states for each of these combinations are represented on the main diagonal of an Hermitian matrix.

<img src="img\max_cut_mat.png" width="290 px" align="center">

&nbsp;

To obtain the energy value for the basis state $|011\rangle$, representing the coloring of the first figure, we have to apply the Hermitian matrix to the vector describing the state:

<img src="img\max_cut_app.png" width="350 px" align="center">

&nbsp; 

Since applying the operator to $|011\rangle$ yields the same state multiplied by a scalar, it means the state is an **eigenstate** of the operator, with a corresponding **eigenvalue** of -1. To obtain the energy value as a scalar, we apply the inner product of it with the row vector representing the state:

<img src="img\innerp.png" width="290 px" align="center">

&nbsp;

which may also be represented in Dirac notation as $\langle 011 | Q | 011 \rangle = -1$. For a general state, $\langle \psi | Q | \psi \rangle$ is known as the **expectation value** of $Q$ the state $|\psi \rangle$.

The solution to the $MaxCut$ problem is the basis state for which the operator Q has the lowest expectation value.

&nbsp;


### The problem

&nbsp;

The Variational Quantum Eigensolver is designed to solve the following problem:

$$ \min_{| \psi \rangle} \; \langle \psi | Q | \psi \rangle $$

where $| \psi \rangle \in (\mathbb{C}^2)^{\otimes n}$, and $ Q \in \mathbb{C}^{2^n \times 2^n}$ and Hermitian. The operator $Q$ is typically called the Hamiltonian and denoted by $H$.

This problem is difficult to solve, since the vector representing $|\psi\rangle$ has dimension $2^n$ (indeed, it is NP-hard). A way to reduce the dimensionality of the problem leads to the VQE algorithm: rather than optimizing over all possible states, we only optimize over a set of states $ |\psi (\vec{\theta})\rangle = U (\vec{\theta}) | \vec{0} \rangle $.

The problem solved by VQE then becomes:

$$ \min_{\vec{\theta}} \langle \psi (\vec{\theta})| Q | \psi (\vec{\theta}) \rangle$$

The choice of $U (\vec{\theta})$ is fundamental, as it determines the quality of the approximation $ |\psi \rangle \approx |\psi (\vec{\theta}) \rangle$ and the ease of optimizing over $\vec{\theta}$ (how many $\theta_i$ do we optimize?). The circuit implementing $U(\vec{\theta})$ is usually called *ansatz*.

&nbsp;

### The algorithm

&nbsp;

The VQE algorithm uses a quantum subroutine inside a classical minimization task. The quantum subroutine consists of two parts:

* Prepare a quantum state $|\psi (\vec{\theta})\rangle$;
* Measure the expectation value of $Q$ in the state  $|\psi (\theta)\rangle$:

$$ \langle Q \rangle  _{|\psi (\vec{\theta})\rangle} = \langle \psi (\vec{\theta}) | Q | \psi (\vec{\theta}) \rangle $$

The [variational theorem](https://en.wikipedia.org/wiki/Variational_method_(quantum_mechanics)) implies that the expectation value of any wavefunction is always greater or equal to the lowest eigenvaluea associated with $Q$:

$$ \langle Q \rangle  _{|\psi (\vec{\theta})\rangle} \geq \lambda_1 $$

The classical optimizer algorithm provides the initial parameters $\vec{\theta}$ to the quantum subroutine, receives the measured expectation value, and tunes the parameters until the expectation value converges to some minimum.

<img src="img\vqe1.png" width="500 px" align="center">

&nbsp;

<div class="alert alert-block alert-info">
    
**Subroutine quantum circuit**

&nbsp;

When constructing a variational form we must balance two opposing goals. Ideally, our $n$ qubit variational form would be able to generate any possible state $|\psi\rangle$ where $|\psi \rangle \in \mathbb{C}^N$ and $N=2^n$. However, we would like the variational form to use as few parameters as possible. 

For $n=1$, the $U3$ gate takes three parameters and allows for the implementation of any possible state (up to a global phase).

<img src="img\U3_var_form.png" width="300 px" align="center">

In a 2 qubit case, interactions (entanglement) must be considered to to achieve universality. An example of a universal parametrized 2 qubit circuit:

<img src="img\two_qubit_var_form.png" width="500 px" align="center">

Note that variational forms do not *always* have to be universal - they just need to generate all possible states *given the constraints of the problem*, or at least reach a good enough approximation.

One of the challenges in implementing the VQE algorithm is selecting an efficiently parametrized variational form, i.e. one in which the number of parameters does not grow exponentially with the number of qubits.


</div>

## 1.1 Implementation <a id='impl'></a>

**Module Imports**

In [ ]:
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import math

from qiskit import BasicAer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.ising import max_cut, tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising.common import sample_most_likely

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

#from qiskit import IBMQ
#provider = IBMQ.load_account()

**Generating the graph for the Max-Cut problem**

In [ ]:
# Generating a graph of 4 nodes 

n=4 # Number of nodes in graph
G=nx.Graph()
G.add_nodes_from(np.arange(0,n,1))
elist=[(0,1,1.0),(0,2,1.0),(0,3,1.0),(1,2,1.0),(2,3,1.0)]

# tuple is (i,j,weight) where (i,j) is the edge
G.add_weighted_edges_from(elist)

colors = ['r' for node in G.nodes()]
pos = nx.spring_layout(G)
default_axes = plt.axes(frameon=True)
nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)

In [ ]:
# Computing the weight matrix from the random graph
w = np.zeros([n,n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i,j,default=0)
        if temp != 0:
            w[i,j] = temp['weight'] 
print(w)

<div class="alert alert-block alert-info">
    
**Creating a Hamiltonian from the adjacency matrix**

&nbsp;

There are $2^n$ energy states in our Hamiltonian (more specifically an Ising Hamiltonian), where $n$ is the number of vertices in the graph. These energy states are computed from the weights on the graph’s edges, which are expressed in the adjacency matrix. To accomplish this, Qiskit Aqua leverages the Pauli Z matrix and the identity matrix.

<img src="img\adj_mat.png" width="500 px" align="center">

Each edge can be represented with a pair of Pauli Z matrices for the connected vertices; vertices not connected by a particular edge are modeled with the identity matrix. We then take the tensor product of all vertices, scale the matrix by $\frac{1}{2}$ and the weight $w_{ij}$.

&nbsp;

**Example**: for the edge that connects vertices $A$ and $C$ in the figure above:

<img src="img\adj_mat2.png" width="300 px" align="center">

After all edges are modelled, the corresponding matrices are added together. The general expression to compute the Ising Hamiltonian from an adjacency matrix is:

&nbsp;

$$ \frac{1}{2} \left ( \sum_{i < j} w_{ij} Z_i Z_j \right ) $$

&nbsp;

</div>



**Brute force approach**

In [ ]:
best_cost_brute = 0
for b in range(2**n):
    x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]
    cost = 0
    for i in range(n):
        for j in range(n):
            cost = cost + w[i,j]*x[i]*(1-x[j])
    if best_cost_brute < cost:
        best_cost_brute = cost
        xbest_brute = x 
    print('case = ' + str(x)+ ' cost = ' + str(cost))

colors = ['r' if xbest_brute[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, pos=pos)
print('\nBest solution = ' + str(xbest_brute) + ' cost = ' + str(best_cost_brute))    

In [ ]:
# Maping to an Ising Hamiltonian 
qubitOp, offset = max_cut.get_operator(w)

In [ ]:
#Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = ExactEigensolver(qubitOp, k=1)
result = ee.run()

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

colors = ['r' if max_cut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha = .8, pos=pos)

**Running it on a quantum computer**

In [ ]:
# run quantum algorithm with shots
seed = 10598

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))
plot_histogram(result['eigvecs'][0])

colors = ['r' if max_cut.get_graph_solution(x)[i] == 0 else 'b' for i in range(n)]
nx.draw_networkx(G, node_color=colors, node_size=600, alpha = .8, pos=pos)

&nbsp;

<div class="alert alert-block alert-warning">
    
**Exercise 1**

&nbsp;

1. Modify the implementation to apply the VQE algorithm to solve the $MaxCut$ problem to the graph below, providing a coloring solution.


<img src="img\graph-1.png" width="400 px" align="center">



</div>



&nbsp;

# 2. Quantum Approximate Optimization Algorithm (QAOA) <a id='QAOA'></a>

&nbsp;

The quantum approximate optimization algorithm (QAOA) is an algorithm proposed to solve combinatorial optimization problems [Farhi et al., 2014](https://arxiv.org/abs/1411.4028).  One of its main features is the ability to  control  the  depth  of  the  circuits  required  by  the  algorithm,  so  that  in  principle  one  can restrict the algorithm to shallow circuits, which are more likely to be successfully implemented on the current generation of quantum hardware.

The QAOA can be seen as a form of VQE with a specific choice of the variational form that is derived from the problem Hamiltonian $H$.  QAOA enjoys somewhat stronger convergence properties than VQE.

Given an Hamiltonian of the same form as defined in the VQE problem, the variational form of QAOA is constructed with a layer of Hadamard gates, followed by two alternating unitaries:

$U_C(\gamma) = e^{-i \gamma} \; ( \; \sum_{j=1}^n c_j \sigma^Z_j + \sum_{j,k=1}^n C_{jk} \sigma^Z_j \otimes \sigma_k^Z \; )$

$U_B(\beta) = e^{-i \beta} \sum_{j=1}^n \sigma_j^X$

where $\sigma_j^Z$ denotes the Pauli $Z$ matrix acting on the $j$-th qubit, and $\sigma_j^X$ denotes the Pauli $X$ matrix acting on the $j$-th qubit. For a given depth parameter $p \in \mathbb{N}$, the variational form is thus defined as

$$ U(p,\beta,\gamma) = \left [ \prod_{j=1}^p U_B(\beta_j) U_C(\gamma_j) \right ] \; H^{\otimes n}$$

The variational form $|\psi (p, \beta, \gamma) \rangle = U (p, \beta, \gamma) | \vec{0} \rangle$ thus replaces $|\psi (\vec{\theta})\rangle$ discussed in the previous section, and the computation and optimization of the measured expectation value proceed in the same way.

It is possible to construct much simples variational forms that also have the property of yielding the optimum if correctly parametrized. A commonly used variational form is composed of single qubit rotation $R_y$ gates:

$$ U(\vec{\theta}) = R_y (\theta_1) \otimes R_y (\theta_2) \otimes ... \otimes R_y(\theta_n) $$

Since $R_y(0) |0\rangle = |0\rangle$ and $R_y(\frac{\pi}{2}) |0\rangle = |1\rangle$, with angle $\theta_j \in \{0, \frac{\pi}{2} \} $ this variational form spans all basis states, and is guaranteed to span the optimal solution of diagonal Hamiltonians such as the one demonstrated in the $MaxCut$ example.



### Additional resources:

[Qiskit tutorial for simulation of molecule ground states with VQE](https://qiskit.org/textbook/ch-applications/vqe-molecules.html)

[Qiskit tutorial of combinatorial optimization problems with QAOA](https://qiskit.org/textbook/ch-applications/qaoa.html)